# Apply the trained model to data

1. Apply model to get the average drug use in the entire region  
2. Apply model to data that is geolocated and look at the geographic distribution of drug use

## Import packages and set directory

In [2]:
import autogluon
import pandas as pd
import pathlib
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

from autogluon.tabular import TabularDataset, TabularPredictor

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

repo_f = pathlib.Path.cwd().parent.parent
code_f = repo_f.joinpath('code')
input_f = repo_f.joinpath('input')
output_f = repo_f.joinpath('output')


c:\Users\JosephNaddeo\anaconda3\envs\mdi_workshop_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_census = pd.read_csv(input_f.joinpath("census\sc_pums_harmonized.csv"))
try:
    df_census.drop(columns=['Unnamed: 0'] , inplace=True)

except:
    pass
# create index for df to be used at the end

df_census.reset_index(inplace=True, drop=False)


In [4]:
df_census.head()

,index,year,sample,serial,cbserial,hhwt,cluster,statefip,density,metro,met2013,metpop10,pctmetro,puma,strata,gq,pernum,perwt,age_ns,sex_ns,mar_ns,edu_ns,race_ns,emp_ns,pinc_ns,finc_ns,pov_ns
0,0,2015,201501,1093406,157,174,2015010934061,45,1030.20000,4,16700,664607,100,1202,120245,1,1,174,15,1,4,8,1,2,3,3,2
1,1,2015,201501,1093418,786,20,2015010934181,45,416.10001,3,16700,664607,100,1203,120345,1,1,20,16,1,3,8,1,1,4,4,3
2,2,2015,201501,1093419,815,216,2015010934191,45,1030.20000,4,16700,664607,100,1202,120245,1,1,215,16,2,1,8,2,3,1,2,1
3,3,2015,201501,1093419,815,216,2015010934191,45,1030.20000,4,16700,664607,100,1202,120245,1,2,80,16,1,1,8,2,1,2,2,1
4,4,2015,201501,1093419,815,216,2015010934191,45,1030.20000,4,16700,664607,100,1202,120245,1,3,118,12,1,4,6,2,4,1,2,1


In [5]:
## Rename to match model input data
X_full = df_census[['index', 'year', 'age_ns', 'sex_ns',	'mar_ns',	'edu_ns',	'race_ns',	'emp_ns',	'pinc_ns',	'finc_ns',	'pov_ns']]
cat_vars = ['agegrp', 'marstat', 'male', 'edu', 'race', 'employ',	'income',	'famincome',	'poverty']
X_full = X_full.rename(columns={'age_ns':'agegrp', 'mar_ns':'marstat', 'sex_ns':'male', 'edu_ns':'edu', 'race_ns':'race', 'emp_ns':'employ', 'pinc_ns':'income', 'finc_ns':'famincome', 'pov_ns':'poverty'})

## Make integer
for i in cat_vars:
    X_full[i] = X_full[i].astype(int)


# Create dummies
X_full = pd.get_dummies(X_full, columns=cat_vars,drop_first=False, prefix_sep='')
# add poverty==0 column (there were no poverty==0 in census data)
X_full['poverty0'] = 0


In [6]:
## Apply trained model
folder = 'agModels-predictnonmj_use'
path = output_f.joinpath(folder) 

## Get leaderboard to get best model
leaderboard = pd.read_csv(output_f.joinpath('leaderboard_nonmj_use' + '.csv'))
best_model = leaderboard.sort_values('score_test', ascending=False).loc[0]['model']

## Load model and apply it
predictor = TabularPredictor.load(path)
X_full['nonmj_use_prob'] = predictor.predict_proba(X_full, as_multiclass=False, model=best_model)

In [12]:
df_census_analysis = pd.merge(df_census, X_full[['index', 'nonmj_use_prob']], on='index', validate='1:1')
df_census_analysis['non_mj_user'] = np.where(df_census_analysis['nonmj_use_prob'] >= 0.5, 1, 0)

In [13]:
df_census_analysis.head()

,index,year,sample,serial,cbserial,hhwt,cluster,statefip,density,metro,met2013,metpop10,pctmetro,puma,strata,gq,pernum,perwt,age_ns,sex_ns,mar_ns,edu_ns,race_ns,emp_ns,pinc_ns,finc_ns,pov_ns,nonmj_use_prob,non_mj_user
0,0,2015,201501,1093406,157,174,2015010934061,45,1030.20000,4,16700,664607,100,1202,120245,1,1,174,15,1,4,8,1,2,3,3,2,0.023959,0
1,1,2015,201501,1093418,786,20,2015010934181,45,416.10001,3,16700,664607,100,1203,120345,1,1,20,16,1,3,8,1,1,4,4,3,0.996135,1
2,2,2015,201501,1093419,815,216,2015010934191,45,1030.20000,4,16700,664607,100,1202,120245,1,1,215,16,2,1,8,2,3,1,2,1,0.000055,0
3,3,2015,201501,1093419,815,216,2015010934191,45,1030.20000,4,16700,664607,100,1202,120245,1,2,80,16,1,1,8,2,1,2,2,1,0.000039,0
4,4,2015,201501,1093419,815,216,2015010934191,45,1030.20000,4,16700,664607,100,1202,120245,1,3,118,12,1,4,6,2,4,1,2,1,0.000002,0


## Look at patterns in data

In [14]:
## By race
f_x = 'non_mj_user ~ C(race_ns)'

ols_res = smf.ols(formula=f_x, data=df_census_analysis).fit(cov_type='HC3')
ols_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            non_mj_user   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2308.
Date:                Wed, 11 Jan 2023   Prob (F-statistic):               0.00
Time:                        09:38:17   Log-Likelihood:             1.3641e+06
No. Observations:             3871483   AIC:                        -2.728e+06
Df Residuals:                 3871476   BIC:                        -2.728e+06
Df Model:                           6                                         
Covariance Type:                  HC3                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.0353      0.000    301.354      0.000       0.035       0.036
C(race_ns)[T.2]    -0.0193      0.000   -112.077      0.000      -0.020      -0.019
C(race_ns)[T.3]     0.0268      0.003     10.569      0.000       0.022       0.032
C(race_ns)[T.4]     0.0427      0.003     14.031      0.000       0.037       0.049
C(race_ns)[T.5]    -0.0182      0.001    -35.999      0.000      -0.019      -0.017
C(race_ns)[T.6]     0.0034      0.001      5.387      0.000       0.002       0.005
C(race_ns)[T.7]    -0.0065      0.000    -17.119      0.000      -0.007      -0.006
==============================================================================
Omnibus:                  3991482.893   Durbin-Watson:                   0.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        148679423.182
Skew:                           5.496   Prob(JB):                         0.00
Kurtosis:                      31.300   Cond. No.                         23.3
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

## Look at geographic distribution of drug use

Using RTI synthpop data, apply our model and look at how drug use is distributed by block group.  

Note: This data is from the 2011-2007 5 year ACS estimates and the model is trained with 2019 survey data.

In [16]:
df_rti = pd.read_parquet(input_f.joinpath('census', 'pums_w_safegraph.parquet'))

In [21]:
## Apply trained model
folder = 'agModels-predictnonmj_use'
path = output_f.joinpath(folder) 

## Get leaderboard to get best model
leaderboard = pd.read_csv(output_f.joinpath('leaderboard_nonmj_use' + '.csv'))
best_model = leaderboard.sort_values('score_test', ascending=False).loc[0]['model']

## Load model and apply it
predictor = TabularPredictor.load(path)
df_rti['nonmj_use_prob'] = predictor.predict_proba(df_rti, as_multiclass=False, model=best_model)

In [22]:
df_rti.head()

,stcotrbg,serialno,sporder,agegrp0,agegrp1,agegrp10,agegrp11,agegrp12,agegrp13,agegrp14,agegrp15,agegrp16,agegrp17,agegrp2,agegrp3,agegrp4,agegrp5,agegrp6,agegrp7,agegrp8,agegrp9,marstat0,marstat1,marstat2,marstat3,marstat4,male1,male2,edu0,edu1,edu10,edu11,edu2,edu3,edu5,edu6,edu7,edu8,edu9,race1,race2,race3,race4,race5,race6,race7,employ0,employ1,employ2,employ3,employ4,income0,income1,income2,income3,income4,income5,income6,income7,famincome0,famincome1,famincome2,famincome3,famincome4,famincome5,famincome6,famincome7,poverty0,poverty1,poverty2,poverty3,edu4,std,median,min,max,mean,_merge,nonmj_use_prob
0,450070114021,2.007001e+12,4.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,left_only,0.000037
1,450070114021,2.007001e+12,1.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,left_only,0.002415
2,450070118003,2.007001e+12,2.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,left_only,0.022621
3,450070110012,2.007001e+12,2.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,left_only,0.022621
4,450070114014,2.007001e+12,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,left_only,0.002222


## Exercise: Look at how patterns correlate with census characteristics

1. Go to https://www.nhgis.org/ and make an account. Then pull data at the block group (or coarser) level for SC.  
2. Aggregate df_rti to block group level. Note: make  decision on how you will aggregate (eg sum vs mean vs median etc)
3. Merge census data from NHGIS onto the aggregated RTI data 
4. Look for interesting correlations!  

Bonus: Use data to make maps to visualize distribution of drug use in state.